# MD postprocessing

##### This notebook requires MDanalysis Matplotlib NGLview Pandas and Numpy

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import MDAnalysis as mda
from MDAnalysis.analysis import rms

import nglview as nv

### Input parameters

Define input parameters and paths

In [ ]:
replicate = 'rep4'

path_data = 'MD_trajectories/'+replicate+'/'

#Input files
pdb = path_data + 'conf.gro'
trajectory = path_data + 'VMD_aligned.dcd'
reference_system = path_data + 'conf.gro'
path = path_data + 'analysis/'

#MD timestep
timestep = 2 #femtoseconds

#Selections
ligand_selection = "resname LIG"
protein_selection = "protein"
backbone_selection = "backbone"
protein_ligands_selection = "protein or resname LIG NAP"

### Detect binding site residues

In [ ]:
# Load your .gro file into an MDAnalysis Universe
u = mda.Universe(pdb)

# Select the ligand atoms by residue name
ligand = u.select_atoms('resname LIG')

# Select protein residues 
binding_site_residues = u.select_atoms(f'protein and resid 111 112 113 116 124 125 128 132 153 170 174 200', ligand=ligand)

# Extract unique residue IDs
binding_site_resids = binding_site_residues.residues.resids

### Load trajectory

In [ ]:
# Load trajectory
u = mda.Universe(pdb, trajectory)

# Select the protein atoms
system_atoms = u.select_atoms(protein_ligands_selection)

### Extract RMSF of binding site residues (side chains and backbone)

In [ ]:
# Select the protein atoms
protein_atoms = u.select_atoms('protein')

# Calculate RMSF for the whole sidechain of specific residues
rmsf_values = np.zeros(len(binding_site_resids))
x_labels = []  # To store x-axis labels with residue names and IDs

for i, resid in enumerate(binding_site_resids):
    residue_atoms = protein_atoms.select_atoms(f"resid {resid}")
    residue_name = residue_atoms.residues.resnames[0]  # Get the residue name
    R = rms.RMSF(residue_atoms).run()
    rmsf_values[i] = np.mean(R.rmsf)
    
    # Create the label with residue name and ID
    x_labels.append(f"{residue_name}-{resid}")

# Plot the barplot
plt.figure(figsize=(11, 3))
plt.bar(x_labels, rmsf_values, color='dodgerblue', align='center')
plt.xlabel("Residue (Name-ID)", fontsize=14)
plt.ylabel("Average RMSF (Å)", fontsize=14)
#plt.title("Root-Mean-Square Fluctuations per Residue of Binding Site Residues")
plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility
plt.grid(axis='y')  # Only show horizontal grid lines
plt.ylim(0, 3.2)
plt.yticks(size=14)
plt.xticks(size=14)
plt.savefig(path + 'rmsf_binding_site.png')
plt.show()


### NGL representation of binding site residues

Shows binding site residues based on the previous computed RMSF

In [ ]:
# Set B-factors as tempfactors for visualization
u.add_TopologyAttr('tempfactors')  # add empty attribute for all atoms

# Convert binding_site_resids list to string for selection
string_binding_site = ' '.join(str(num) for num in binding_site_resids)

# Select atoms in the binding site
binding_site_sel = u.select_atoms('resid ' + string_binding_site)

# Assign RMSF values to B-factors (tempfactors)
for residue, r_value in zip(binding_site_sel.residues, rmsf_values):
    residue.atoms.tempfactors = r_value

# Select protein atoms in the binding site and ligand for visualization
binding_site_sel_and_ligand = u.select_atoms('resid ' + string_binding_site + ' or ' + ligand_selection)

# Visualize the binding site and ligand using NGLView
view = nv.show_mdanalysis(binding_site_sel_and_ligand)
view.clear()
view.add_representation("licorice", selection='not protein', radius=0.6)

# Visualize each residue in the binding site with B-factors as licorice representation
for resid_bs in binding_site_resids:
    residue_selection = 'protein and resid ' + str(resid_bs)
    view.add_licorice(color_scheme='bfactor', selection=residue_selection)
    
print('Darker colors means higher RMSF')

# Display the visualization
view

### RMSD of Binding site

Computes the RMSD. Does the aligment on the backbone

In [ ]:
#LOADING FILES
ref = mda.Universe(reference_system) 

#RMSD OF A TRAJECTORY USING MULTIPLE SELECTIONS
string_binding_site = ' '.join(str(num) for num in binding_site_resids)
binding_site = 'resid '+string_binding_site

R = rms.RMSD(u,  # universe to align
             u,  # reference universe or atomgroup
             select=binding_site,  # group to superimpose and calculate RMSD
             groupselections=[binding_site,],  # groups for RMSD
             ref_frame=0,  # frame index of the reference
             superposition=False)  # skip alignment
R.run()


# STORE DATA IN TABLE
df = pd.DataFrame(R.results.rmsd, columns=['Frame', 'Time (ns)', 'binding_site', 'binding_site'])

df['Time (ns)'] = df['Time (ns)'] * 10

# PLOTTING THE DATA IN GRAPH
plt.figure(figsize=(10, 4))

# Plot the RMSD binding_site in the same plot with different colors and labels
plt.plot(df['Time (ns)'], df['binding_site'], color='lightblue', label='Binding Site', linewidth=1)

# Add a legend to differentiate between the lines
plt.legend()

# Set the labels and title
plt.xlabel('Time (ns)')
plt.ylabel(r'RMSD ($\AA$)')
plt.title('Root-Mean-Square Deviations (RMSD)')

# Display the plot
plt.grid(True)
plt.savefig(path + 'rmsd.png')
plt.show()

#save the data in a csv file
df.to_csv(path+'rmsd.csv', index=False)


### 5 replicates RMSD plot

In [ ]:
# Define replicates
replicates = ['rep1', 'rep2', 'rep3', 'rep4', 'rep5']

# Set up the figure size
plt.figure(figsize=(12, 6))

# Define a color palette with shades of blue
colors = ['darkviolet', 'darkblue', 'aquamarine', 'dodgerblue', 'purple']  # Pink, purple, blue, gray, and black
line_styles = ['-', '-', '-', '-', '-']  # Different line styles

# Loop through replicates and plot RMSD
for i, rep in enumerate(replicates):
    df = pd.read_csv(f'MD_trajectories/{rep}/analysis/rmsd.csv')
    plt.plot(df['Time (ns)'], df['binding_site'], label=rep, color=colors[i], linestyle=line_styles[i], linewidth=1.5)

# Improve plot aesthetics
plt.xlabel('Time (ns)', fontsize=14)
plt.ylabel(r'RMSD ($\AA$)', fontsize=14)
plt.title('Root-Mean-Square Deviations of the Binding Site Residues', fontsize=16, weight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(visible=True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
plt.legend(title='Replicates', fontsize=12, title_fontsize=13, loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=len(replicates))
plt.tight_layout()

plt.savefig('MD_trajectories/rmsd_replicates.png')
# Show the plot
plt.show()

